Before running the next cell please add a shortcut to the shared folder at the root of your Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/Learning_with_Density_Matrices_and_Random_Features-Supplementary_Material/qmc/tf')
import layers as layers
import models as models

/usr/local/lib/python3.6/dist-packages/typeguard/__init__.py:804: UserWarning: no type annotations present -- not typechecking layers.CrossProduct.__init__
  warn('no type annotations present -- not typechecking {}'.format(function_name(func)))
/usr/local/lib/python3.6/dist-packages/typeguard/__init__.py:804: UserWarning: no type annotations present -- not typechecking layers.DensityMatrix2Dist.__init__
  warn('no type annotations present -- not typechecking {}'.format(function_name(func)))
/usr/local/lib/python3.6/dist-packages/typeguard/__init__.py:804: UserWarning: no type annotations present -- not typechecking layers.DensityMatrixRegression.__init__
  warn('no type annotations present -- not typechecking {}'.format(function_name(func)))


In [ ]:
import warnings
import numpy as np
from sklearn.kernel_approximation import RBFSampler

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
import numpy as np
import pylab as pl
import pandas as pd
import gzip

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
import pickle

In [ ]:
import sklearn
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import keras
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import make_scorer

In [ ]:
# from functions.keras_wrapper_qmc import KerasClassifier

# from sklearn.model_selection import RandomizedSearchCV, KFold
# from sklearn.metrics import make_scorer

### example

In [ ]:
!gdown --id 1xYDnWHxRfYINNOpa9iMf4XL_jTiyzKkY
!gdown --id 1BC9gAHgjIMQF4jC11YzV0rtyFewlFsx9
!gdown --id 11KpZtTdd6DRRZ76kU5d8iESiUaPmVPoe
!gdown --id 1IK_uBHrXnx092etVs9us3UbekzSu4IMd
!gdown --id 10X-81KboXdPfwDlZcYkwMflAWCT3memv

Downloading...
From: https://drive.google.com/uc?id=1xYDnWHxRfYINNOpa9iMf4XL_jTiyzKkY
To: /content/gisette_test.data
73.4MB [00:00, 120MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1BC9gAHgjIMQF4jC11YzV0rtyFewlFsx9
To: /content/gisette_train.data
67.8MB [00:00, 88.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=11KpZtTdd6DRRZ76kU5d8iESiUaPmVPoe
To: /content/gisette_train.labels
100% 15.0k/15.0k [00:00<00:00, 11.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1IK_uBHrXnx092etVs9us3UbekzSu4IMd
To: /content/gisette_valid.data
11.3MB [00:00, 69.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=10X-81KboXdPfwDlZcYkwMflAWCT3memv
To: /content/gisette_valid.labels
100% 2.50k/2.50k [00:00<00:00, 10.6MB/s]


In [ ]:
num = 6000
    
train_data = pd.read_csv("gisette_train.data", header=None, sep=" ").iloc[0:num,:5000]
train_labels = pd.read_csv("gisette_train.labels", header=None, sep=" ").iloc[0:num]
test_data = pd.read_csv("gisette_valid.data", header=None, sep=" ").iloc[0:num,:5000]
test_labels = pd.read_csv("gisette_valid.labels", header=None, sep=" ").iloc[0:num]


print(train_data.head())
print(train_data.describe())
print(train_data.info())
print(np.unique(train_labels))
print(train_data.shape)

train_labels[train_labels == -1] = 0
test_labels[test_labels == -1] = 0



   0     1     2     3     4     5     ...  4994  4995  4996  4997  4998  4999
0   550     0   495     0     0     0  ...   991     0     0     0     0   983
1     0     0     0     0     0     0  ...     0     0   991     0     0     0
2     0     0     0     0     0     0  ...     0     0     0     0     0     0
3     0     0   742     0     0     0  ...     0     0   674     0     0   838
4     0     0     0     0     0     0  ...     0     0     0     0     0     0

[5 rows x 5000 columns]
              0            1     ...         4998         4999
count  6000.000000  6000.000000  ...  6000.000000  6000.000000
mean    111.022833    11.251000  ...   122.423500   153.636000
std     284.848291    91.448864  ...   294.247413   326.609648
min       0.000000     0.000000  ...     0.000000     0.000000
25%       0.000000     0.000000  ...     0.000000     0.000000
50%       0.000000     0.000000  ...     0.000000     0.000000
75%       0.000000     0.000000  ...     0.000000     0.0000

In [ ]:
X_train = train_data.values
X_test = test_data.values
y_train = train_labels.values
y_test = test_labels.values

y_train = y_train.astype('int32')
y_test = y_test.astype('int32')

In [ ]:
# Extracting a stratified samples such that we can do a dimensionality reduction
# percentage_for_stratified_samples = 0.95
# random_state = 42

# X_train, _, y_train, _ = train_test_split(X_train, y_train, test_size=percentage_for_stratified_samples, random_state=random_state, stratify=y_train)
# X_test, _, y_test, _ = train_test_split(X_test, y_test, test_size=percentage_for_stratified_samples, random_state=random_state, stratify=y_test)

In [ ]:
# Transforming each feature into a 0 to 1 range
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
gammas = [2**i for i in range(-10,10)]

In [ ]:
# Sanity check for shape of data
print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test: ", y_test.shape)
print("y_train unique: ", np.unique(y_train))
print("y_test unique: ", np.unique(y_test))

X_train:  (6000, 5000)
y_train:  (6000, 1)
X_test:  (1000, 5000)
y_test:  (1000, 1)
y_train unique:  [0 1]
y_test unique:  [0 1]


In [ ]:
from sklearn.kernel_approximation import RBFSampler
rbf_feature = RBFSampler(gamma=2**-9, n_components = 1000)
X_train_features = rbf_feature.fit_transform(X_train)
X_test_features = rbf_feature.transform(X_test)

from sklearn.svm import LinearSVC

model_lsvc = LinearSVC(C=2**1,tol=1e-05)
model_lsvc.fit(X_train_features, y_train)
print(model_lsvc.score(X_test_features, y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.751


In [ ]:
input_dim = X_train.shape[1]
num_classes = np.unique(y_train).shape[0]
gamma = 2**-5
component_dim = 1000
random_state=0
num_eig=0
batch_size=8
epochs = 10

In [ ]:
# Create models from qmkdc classifier

def create_model(input_dim, num_classes, component_dim=100, gamma=1, lr=0.01, decay=0.,
                  random_state=None, eig_percentage=0, initialize_with_rff=False,
                  type_of_rff="rff", fix_rff=False):
    '''This is a model generating function so that we can search over neural net
    parameters and architecture'''

    num_eig = round(eig_percentage * component_dim)

    opt = keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999, decay=decay)

    if type_of_rff == 'rff':
        fm_x = layers.QFeatureMapRFF(input_dim, dim=component_dim, gamma=gamma, random_state=random_state)
    #else:
    #    fm_x = layers.QFeatureMapORF(input_dim, dim=component_dim, gamma=gamma, random_state=random_state)

    if initialize_with_rff:
        qmkdc = models.QMKDClassifier(fm_x=fm_x, dim_x=component_dim, num_classes=num_classes)
        qmkdc.compile()
        qmkdc.fit(X_train, y_train, epochs=1, batch_size=batch_size, verbose=20)

    qmkdc1 = models.QMKDClassifierSGD(input_dim=input_dim, dim_x=component_dim, num_eig=num_eig,
                                      num_classes=num_classes, gamma=gamma, random_state=random_state, fm_x=fm_x)

    if fix_rff:
        qmkdc1.layers[0].trainable = False

    qmkdc1.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=['accuracy'])

    if initialize_with_rff:
        qmkdc1.set_rhos(qmkdc.get_rhos())

    # qmkdc1.fit(X_train, y_train_bin, epochs=epochs, batch_size=batch_size)

    return qmkdc1

In [ ]:
# Tensorboard 
%load_ext tensorboard
%tensorboard --logdir "/gdrive/My Drive/Universidad/doctorado_programacion/logs/"

In [ ]:
# Creation of the model without random search. It can be used to verify the initial performance of the model.
import datetime, os
logdir = os.path.join("/gdrive/My Drive/Universidad/doctorado_programacion/logs", "letters-" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

model_mnist = create_model(input_dim, num_classes, component_dim=1000, gamma=2**-9, lr=1e-04, decay=0.,
                  random_state=None, eig_percentage=1, initialize_with_rff=True,
                  type_of_rff="rff", fix_rff=False)

y_train_bin = tf.reshape(tf.keras.backend.one_hot(y_train, num_classes), (-1, num_classes))
y_test_bin = tf.reshape(tf.keras.backend.one_hot(y_test, num_classes), (-1, num_classes))
model_mnist.fit(X_train, y_train_bin.numpy(), epochs=50, batch_size=batch_size, validation_data=(X_test, y_test_bin.numpy()), 
            callbacks=[tensorboard_callback])

In [ ]:
# Final validation test
y_test_bin = tf.reshape(tf.keras.backend.one_hot(y_test, num_classes), (-1, num_classes))
out = model_mnist.predict(X_test)
accuracy_score(y_test, out.argmax(axis=1))

0.72

In [ ]:
# Creation of KerasClassifier for sending to random search
model = KerasClassifier(build_fn=create_model, epochs=1, batch_size=16, verbose=1)

In [ ]:
# component dimension
from scipy.stats import randint
#components_dimensions = randint(20,1500)
components_dimensions = [1000]

# gamma
#gammas = [2 ** i for i in range(-25, 25)]
gammas = [2 ** i for i in range(4, 6)]

# number of eigen values
eig_values = [1 / 10, 25 / 100, 50 / 100, 1]

# initialize with rff
#initialize_with_rff = [True, False]
initialize_with_rff = [True]

# type_of_rff_components
type_of_rff = ['rff', 'orf']

# fix_rff
fix_rff = [True]

# learning algorithm parameters
lr = [1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7]
decay = [1e-6, 1e-9, 0]

# dictionary summary
param_grid = dict(input_dim=[input_dim], num_classes=[num_classes], component_dim=components_dimensions, gamma=gammas,
                  lr=lr, decay=decay,
                  random_state=[None], eig_percentage=eig_values, initialize_with_rff=initialize_with_rff,
                  type_of_rff=type_of_rff, fix_rff=fix_rff
                  )

In [ ]:
grid = RandomizedSearchCV(estimator=model, cv=KFold(3), param_distributions=param_grid,
                              verbose=20, n_iter=1, n_jobs=1)

In [ ]:
# Si no tenemos problema con colab utilizar el siguiente comando
# grid_result = grid.fit(X_train, y_train)
y_train_bin = tf.reshape(tf.keras.backend.one_hot(y_train, num_classes), (-1, num_classes)).numpy()

In [ ]:
cv_results_df = None
n_experiments = 25

for i in range(n_experiments):
  grid_result = grid.fit(X_train, y_train_bin)

  if cv_results_df is None:
    cv_results_df = pd.DataFrame(grid_result.cv_results_)
  else:
    cv_results_df = pd.concat([cv_results_df, pd.DataFrame(grid_result.cv_results_)])

  cv_results_df.to_csv('/gdrive/My Drive/qmkdc gisette random search.csv')

In [ ]:
experiments = pd.read_csv("/gdrive/My Drive/gisette.csv")

In [ ]:
experiments.groupby(["experiment", "gamma", "initialize_rhos", "lr", "eig_percentage", "type_of_rff", "fix_rff", "sgd"]).agg({"mean","std", "count"}).reset_index()

experiment       gamma  ... current_experiment      
                                                ...                std count
0                          qmc_rff    0.007812  ...           1.000000     3
1                      qmc_rff_sgd    0.007812  ...           0.462125    60
2                      qmc_rff_sgd    0.015625  ...           0.000000    35
3                      qmc_rff_sgd    0.031250  ...           1.357922    59
4                      qmc_rff_sgd    0.062500  ...           1.441722   182
5                      qmc_rff_sgd    0.125000  ...           0.000000     9
6                      qmc_rff_sgd    0.250000  ...           1.104532   193
7                      qmc_rff_sgd    0.500000  ...           1.177863   154
8                      qmc_rff_sgd    2.000000  ...           0.498012    86
9                      qmc_rff_sgd    4.000000  ...           1.149185   150
10                     qmc_rff_sgd    8.000000  ...           1.132347    99
11                     qmc_rff_sgd   16.000000  ...           0.000000    48
12  qmc_rff_sgd_wo_initialize_rhos    0.007812  ...           1.417051   250
13  qmc_rff_sgd_wo_initialize_rhos    0.015625  ...           1.100333   192
14  qmc_rff_sgd_wo_initialize_rhos    0.031250  ...           1.411255   248
15  qmc_rff_sgd_wo_initialize_rhos    0.062500  ...           1.308120   213
16  qmc_rff_sgd_wo_initialize_rhos    0.125000  ...           0.835643   132
17  qmc_rff_sgd_wo_initialize_rhos    0.250000  ...           1.972800   105
18  qmc_rff_sgd_wo_initialize_rhos    0.500000  ...           1.443376   241
19  qmc_rff_sgd_wo_initialize_rhos    1.000000  ...           1.229419    39
20  qmc_rff_sgd_wo_initialize_rhos    2.000000  ...           1.661114   110
21  qmc_rff_sgd_wo_initialize_rhos    4.000000  ...           1.417051   250
22  qmc_rff_sgd_wo_initialize_rhos    8.000000  ...           1.695853   158
23  qmc_rff_sgd_wo_initialize_rhos   16.000000  ...           1.217882   110
24  qmc_rff_sgd_wo_initialize_rhos   32.000000  ...           1.337021   212
25  qmc_rff_sgd_wo_initialize_rhos   64.000000  ...           0.586009   105
26  qmc_rff_sgd_wo_initialize_rhos  128.000000  ...           0.998379   169

[27 rows x 29 columns]